In [ ]:
import warnings
import json
import subprocess

from omegaconf import DictConfig
from phalp.utils import get_pylogger
from PHALP_FAST import PHALP

warnings.filterwarnings("ignore")
log = get_pylogger(__name__)

# load config
cfg_fp = "/mnt/arc/levlevi/nba-positions-videos-dataset/4d-pose-extraction/PHALP/PHALP/scripts/config.json"
with open(cfg_fp, "r") as f:
    cfg = DictConfig(json.load(f))

# load PHALP obj
phalp_tracker = PHALP(cfg)

In [20]:
import os
from glob import glob

annotations_dir = "/mnt/arc/levlevi/nba-positions-videos-dataset/nba-plus-statvu-dataset/filtered-clip-annotations"
all_annotation_file_paths = sorted(glob(annotations_dir + "/*/*/*.json"))

ex_fp = all_annotation_file_paths[0]
ex_vid_fp = ex_fp.replace("filtered-clip-annotations", "filtered-clips").replace("_annotation", "").replace(".json", ".mp4")
assert os.path.exists(ex_vid_fp), f"{ex_vid_fp} doesn't exist"

In [21]:
ex_vid_fp

'/mnt/arc/levlevi/nba-positions-videos-dataset/nba-plus-statvu-dataset/filtered-clips/162142/period2/162142_period2_1+_76939328_0.mp4'

In [ ]:
from concurrent.futures import ProcessPoolExecutor

# launch 8 processes w/ export CUDA_VISIBLE_DEVICES={DEVICE} and a subset of file_paths
# 

In [18]:
import json
import numpy as np
import os
import joblib 
import pickle
import sys

from typing import List
from glob import glob

BASE_PATH = ""

def save_data(data, output_filename):
    """Save the processed data to a pickle file using joblib."""
    joblib.dump(data, output_filename)

def process_bbx(bbx, scale_factor=1.075):
    track_id = int(bbx['player_id'])
    x1, y1, width, height = bbx['x'], bbx['y'], bbx['width'], bbx['height']
    scaled_width = width * scale_factor
    scaled_height = height * scale_factor
    # adjust x1, y1 to maintain the center of the bbox
    x1 = x1 - (scaled_width - width) / 2
    y1 = y1 - (scaled_height - height) / 2
    # create the bbox in the format x1, y1, width, height
    bbox_arr = np.array([x1, y1, scaled_width, scaled_height], dtype=np.float32)
    return track_id, bbox_arr

def convert_annotations_to_pkl(src_fp:str, dst_fp:str):
    """
    Convert an annotation file to pkl bbxs.
    """

    data = {}
    with open(src_fp, 'r') as f:
        annotations = json.load(f)
    for frame in annotations['frames']:
        frame_id = int(frame['frame_id'])
        bbxs = frame['bbox']
        for bbx in bbxs:
            track_id, bbox_arr = process_bbx(bbx)
            frame_name = os.path.join(BASE_PATH, f'{frame_id:06}.jpg')
            if frame_name not in data:
                data[frame_name] = {"gt_bbox": [], "extra_data": {"gt_class": [], "gt_track_id": []}}
            data[frame_name]["gt_bbox"].append(bbox_arr.tolist())  # Convert bbox to list for JSON-like output
            if track_id not in data[frame_name]["extra_data"]["gt_track_id"]:
                data[frame_name]["extra_data"]["gt_track_id"].append(track_id)
    save_data(data, dst_fp)
    
dst_fp = "test.pkl"
convert_annotations_to_pkl(ex_fp, dst_fp)